In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4

In [1]:
import argparse
import os
import sys
import time
import numpy as np
import tensorflow as tf
from six.moves import cPickle
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
class model():
    def __init__(self,profile="harmonic"):
        self.x = tf.placeholder(tf.float32, [None, 784])
        self.y = tf.placeholder(tf.int32,[None,10])
        self.learning_rate = 0.001

        self.W1 = tf.get_variable(initializer=tf.random_uniform([784, 100],-0.1,0.1),name="W1")
        self.b1 = tf.get_variable(initializer=tf.random_uniform([100],-0.1,0.1),name="b1")
        self.W2 = tf.get_variable(initializer=tf.random_uniform([100, 100],-0.1,0.1),name="W2")
        self.b2 = tf.get_variable(initializer=tf.random_uniform([100],-0.1,0.1),name="b2")
        self.W3 = tf.get_variable(initializer=tf.random_uniform([100, 10],-0.1,0.1),name="W3")
        self.b3 = tf.get_variable(initializer=tf.random_uniform([10],-0.1,0.1),name="b3")

        if profile == "linear":
            self.r1 = tf.get_variable(initializer=np.linspace(1,0,num=100,endpoint=False,dtype='float32'),name="r1",dtype='float32')
            self.r2 = tf.get_variable(initializer=np.linspace(1,0,num=100,endpoint=False,dtype='float32'),name="r2",dtype='float32')
            self.r3 = tf.get_variable(initializer=np.linspace(1,0,num=10,endpoint=False,dtype='float32') ,name="r3",dtype='float32')
        else:# profile == "harmonic":
            self.r1 = tf.get_variable(initializer=np.array(1.0/(np.arange(100)+1),dtype='float32'),name="r1",dtype='float32')
            self.r2 = tf.get_variable(initializer=np.array(1.0/(np.arange(100)+1),dtype='float32'),name="r2",dtype='float32')
            self.r3 = tf.get_variable(initializer=np.array(1.0/(np.arange(10)+1),dtype='float32') ,name="r3",dtype='float32')

    def train(self,sess,config,gamma_trainable = False):
        rm_vars = [self.r1,self.r2,self.r3]
        # calculate r1*(W1*x)+b
        z1 = tf.add(tf.multiply(self.r1,tf.matmul(self.x,self.W1)),self.b1)
        y1 = tf.nn.relu(z1)
        z2 = tf.add(tf.multiply(self.r2,tf.matmul(y1,self.W2)),self.b2)
        y2 = tf.nn.relu(z2)

        self.logits = tf.add(tf.multiply(self.r3,tf.matmul(y2,self.W3)),self.b3)
        self.probs = tf.nn.softmax(self.logits)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,labels=self.y)
        self.loss = tf.reduce_mean(cross_entropy)

        tvars_trainable = tf.trainable_variables()
        print(tvars_trainable)
        if gamma_trainable:
            print('All variables are trainable.')
        else:
            for rm in rm_vars:
                tvars_trainable.remove(rm)
                print('%s is not trainable'% rm)

        tvars = tf.trainable_variables()
        self.tvars = tvars
        self.tvars_trainable = tvars_trainable
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,var_list=self.tvars_trainable)
        
        for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
            sess.run(v.initializer)

        epochs = config['epochs']
        batch_per_epoch = config['batch_per_epoch']
        batch_size = config['batch_size']
        save_dir = config['save_dir']
        mnist = config['mnist']

        #saver = tf.train.Saver(tf.global_variables())
        loss_to_draw_epoch = []
        for epoch in range(0,epochs):
            # train one epoch
            start_time = time.time()
            train_loss = []
            for batch in range(0,batch_per_epoch):
                batch_x, batch_y = mnist.train.next_batch(batch_size)
                _,loss = sess.run([self.train_op, self.loss], 
                                  feed_dict = {self.x:batch_x, self.y: batch_y})
                train_loss.append(loss)
                #print(loss)

            # test after one epoch

            val_loss , probs = sess.run([self.loss,self.probs],feed_dict={self.x:mnist.test.images, self.y:mnist.test.labels})

            loss_to_draw_epoch.append(val_loss)

            print(" epoch: %4d \t time: %4f \t loss: %4f \t val_loss: %4f" % 
                  (epoch,time.time() - start_time,np.mean(train_loss), val_loss))
        
    def predict(self,sess,idp):
        # t1
        n_ones = int(100 * idp)
        n_zeros = 100 - n_ones
        self.t1 = tf.get_variable(initializer=np.hstack([np.ones(n_ones),np.zeros(n_zeros)]),name="t1")
        
        # t2
        n_ones = int(100 * idp)
        n_zeros = 100 - n_ones
        self.t2 = tf.get_variable(initializer=np.hstack([np.ones(n_ones),np.zeros(n_zeros)]),name="t2")
        
        # t3
        n_ones = int(10 * idp)
        n_zeros = 10 - n_ones
        self.t3 = tf.get_variable(initializer=np.hstack([np.ones(n_ones),np.zeros(n_zeros)]),name="t3")
                
        self.p1 = tf.get_variable(initializer=tf.multiply(self.r1,self.t1),name="p1")
        self.p2 = tf.get_variable(initializer=tf.multiply(self.r2,self.t2),name="p2")
        self.p3 = tf.get_variable(initializer=tf.multiply(self.r3,self.t3),name="p3")
        
        self.rm_vars = [self.r1,self.r2,self.r3,self.t1,self.t2,self.t3,self.p1,self.p2,self.p3]
            
        # calculate r1*(W1*x)+b
        z1 = tf.add(tf.multiply(self.r1,tf.matmul(self.x,self.W1)),self.b1)
        y1 = tf.nn.relu(z1)
        z2 = tf.add(tf.multiply(self.r2,tf.matmul(y1,self.W2)),self.b2)
        y2 = tf.nn.relu(z2)
        
        self.logits = tf.add(tf.multiply(self.r3,tf.matmul(y2,self.W3)),self.b3)
        
        self.probs = tf.nn.softmax(self.logits)
        

In [8]:
model_test = model()

In [5]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
save_dir = ""
epochs = 10
batch_per_epoch = 100
batch_size = 28
config = {'epochs': epochs,
         'batch_per_epoch': batch_per_epoch,
         'batch_size': batch_size,
         'save_dir': save_dir,
         'mnist': mnist}

In [9]:
with tf.Session() as sess:
    model_test.train(sess=sess,config=config,gamma_trainable=False)

AttributeError: 'model' object has no attribute 'train'

In [ ]:
with tf.Session() as sess:
    model_test.train(sess=sess,config=config,gamma_trainable=False)
    print(sess.run(model_test.r1))